In [4]:
!pip install tiktoken

   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   --------------------------------------- 878.7/878.7 kB 13.1 MB/s eta 0:00:00


In [5]:
import tiktoken

In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
with open("the-verdict.txt", "r") as f:
    text = f.read()

enc_text = tokenizer.encode(text)
print(len(enc_text))

5145


In [10]:
enc_sample = enc_text[:50]
print(enc_sample)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]


In [25]:
context_size = 5
for i in range(1,context_size+1):
    context = enc_text[:i]
    target = enc_text[i]

    print(context, "---->", target)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807
[40, 367, 2885, 1464, 1807] ----> 3619


In [27]:
context_size = 5
for i in range(1,context_size+1):
    context = enc_text[:i]
    target = enc_text[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([target]))

I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought
I HAD always thought ---->  Jack


## Implement Data Loader

Step 1: Tokenize the entire text

Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length

Step 3: Return the total number of rows in the dataset

Step 4: Return a single row from the dataset

In [29]:
import torch

from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):

            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

        def __len__(Self):
            return len(self.input_ids)

        def __getitem__(self, idx):
            return self.input_ids[idx], self.target_ids[idx]